In [4]:
from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
import threading
from time import sleep
import pandas as pd
from config import *

class IBapi(EWrapper, EClient):
    def __init__(self, code_index="us500"):
        EClient.__init__(self, self)
        self.df = []
        self.minute = -1
        self.code_list = []
        self._code_index = code_index
        self.conn = None
        self.cursor = None
        self._get_db_connection()

    def _get_db_connection(self):
        self.conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        self.cursor = self.conn.cursor()
        self._get_codelist()
        return None
    
    def _get_codelist(self):
        self.cursor.execute(f"SELECT code from public.maincode where listed = '{self._code_index}';")
        self.conn.commit()
        list_ = self.cursor.fetchall()
        self.code_list = [i[0] for i in list_]
        self.code_list += ["SPY"]
        return None

    def realtimeBar(self, reqId, time, open_, high, low, close, volume, wap, count):
        timestamp_utc = pd.to_datetime(time, unit='s', utc=True)
        tz = 'America/New_York'
        da = timestamp_utc.tz_convert(tz).strftime("%Y-%m-%d %H:%M:%S")
        # print(close, da, reqId)
        code = self.code_list[reqId-1-110]
        if da[-2:] == '00':
            self.df.append([da, code, close])




def run_loop():
    app.run()
app = IBapi()
app.connect('127.0.0.1', 7496, 12321)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
reqId = 111
for symbol in app.code_list[:250]:
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    app.reqRealTimeBars(reqId=reqId, contract=contract, barSize=5, whatToShow="MIDPOINT", useRTH=0, realTimeBarsOptions=[])
    reqId += 1

ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usbond
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


ERROR 115 200 The contract description specified for ABNB is ambiguous.
ERROR 117 456 Max number of real time requests has been reached.
ERROR 118 456 Max number of real time requests has been reached.
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_33013/2919899570.py", line 48, in run_loop
  File "/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/ibapi/client.py", line 263, in run
    self.decoder.i

In [ ]:
import pandas as pd
from datetime import datetime
import threading
import time
from IPython.display import display, clear_output
import psycopg2
# Assuming app and backtest_code are already defined
class ContinuousProcessor:
    def __init__(self):
        self.index_da = None
        self.lock = threading.Lock()
        self.running = True
        
        self.conn = self.get_db_connection()
        self.cursor = self.conn.cursor()
    def get_db_connection(self):
        DB_HOST = 'localhost'
        DB_NAME = 'us'
        DB_USER = 'postgres'
        DB_PASS = 'buddyrich134'
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        return conn
    def backtest_code(self, pivoted, code, rolling=10, benchmark_index='QQQ'):
        try:
            pivoted.ffill()
            ret = pivoted.pct_change(fill_method=None)
            ret['stock_price'] = pivoted[code]
            ret['index_price'] = pivoted[benchmark_index]
            ret['ret_diff'] = ret[code] - ret[benchmark_index]
            # type 2
            ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
            ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
            ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

            # # check direction
            ret['lag10'] = ret['stock_price'].shift(5)
            ret['change_stock'] = (ret['stock_price'] - ret['lag10'])/ret['lag10']
            ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)
            
            # # check if the direction are the same, if not, tned to not being a mean reversion
            ret['lag10_twii'] = ret['index_price'].shift(5)
            ret['change_twii'] = (ret['index_price'] - ret['lag10_twii'])/ret['lag10_twii']
            ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
            ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)
            last = ret.iloc[-1, :]
            da = last.index[-1]
            if last['stdize_ret_diff'] >= 2:
                RED_BACKGROUND = '\033[101m'
                GREEN_BACKGROUND = '\033[102m'
                RESET = '\033[0m'
                color = RED_BACKGROUND if last['direction'] == -1 else GREEN_BACKGROUND
                print(f"{color}{code}, direction:, {last['direction']}, f1: {last['filter_1']}, f2: {last['filter_2']}{RESET}")
                if last['filter_1'] == 1 and last['filter_2'] == 1:
                    self.cursor.execute("INSERT INTO public.block_code3_deatil (da, code, direction, strategy) VALUES (%s, %s, %s, %s)", (da, code, last['direction'], 'intraday_mr'))
                    self.conn.commit()
        except Exception as e:
            print(e, code)
    def continuous_loop(self):
        while self.running:
            df = pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'cl'])
            df = df.pivot(values='cl', columns='code', index='da').sort_index(ascending=True)
            index = df.index[-1]
            if len(df) >= 15 and self.index_da != index:
                self.index_da = index
                clear_output(wait=True)
                display(f'Running analysis for index: {self.index_da}')
                for code in df.columns:
                    if code == "QQQ":
                        continue
                    self.backtest_code(df[['QQQ', code]], code)
                ####
                time.sleep(30)
                pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'cl']).to_parquet("US100.parquet")
                
                self.cursor.execute("SELECT da from public.stock_price order by da desc limit 1;")
                self.conn.commit()
                
                try:
                    da = self.cursor.fetchone()[0]
                    print(da)
                    insert_df = pd.DataFrame(app.df[-500:], columns=['da', 'code', 'cl'])
                    insert_df = insert_df[pd.to_datetime(insert_df['da']) > da]
                except:
                    insert_df = pd.DataFrame(app.df[:], columns=['da', 'code', 'cl'])
                self.cursor.executemany("INSERT INTO public.stock_price (da, code, cl) VALUES (%s, %s, %s);", insert_df.values.tolist())
                self.conn.commit()

                ####

    def stop(self):
        self.running = False

processor = ContinuousProcessor()
loop_thread = threading.Thread(target=processor.continuous_loop, daemon=True)
loop_thread.start()